# Tuto to find What are the collections dand Datasets in Butler

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-04-29
- last update : 2025-05-07 : Add Latiss and LSSTComCamSim

## Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
## Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

### Choose instrument

In [ ]:
instrument = "LSSTCam"
# instrument = "LSSTComCam"
# instrument = "LATISS"
# instrument = "LSSTComCamSim"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LSSTComCamSim":
    repo = "/repo/main"
    collection_validation = instrument + "/raw/all"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "ops_rehersal_prep_2k_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LATISS":
    repo = "/repo/main"
    # collection_validation = instrument + "/runs/quickLook"  # work
    collection_validation = instrument + "/raw/all"
    date_start = 20221001
    date_selection = 20221001
    skymapName = "latiss_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object

In [ ]:
try:
    skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)
except Exception as inst:
    print(type(inst))  # the exception type
    print(inst.args)  # arguments stored in .args
    print(inst)  # __str__ allows args to be printed directly,

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
    if "/calib/" not in _ and "u/" not in _:
        print(_)

In [ ]:
if instrument == "LATISS":
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" in _ or "neveu" in _ or "dagoret" in _:
            print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection_validation).any(execute=False, exact=False):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
        ):
            print(datasetType)